<a href="https://colab.research.google.com/github/debayant9/image_classifier_grayscale_cnn/blob/main/image_classifier_cnn_grayscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.5.0


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [5]:
#Scaling down images in 0 to 1
train_images = train_images/255
test_images = test_images/255

In [6]:
train_images[0].shape

(28, 28)

In [7]:
#how many no of images, pixels count
train_images = train_images.reshape(len(train_images), 28, 28, 1)

In [8]:
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


In [26]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
!pip install keras.tuner

  Using cached https://files.pythonhosted.org/packages/a2/ba/c7a7cda64b67846bde66ffdaaf3199a79d4e35ab6b6f170cc1b7d235646e/keras_tuner-1.0.3-py3-none-any.whl


In [27]:
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [28]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials=5, directory='output', project_name='Mnist')

In [33]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 8 Complete [00h 00m 21s]
val_accuracy: 0.8610000014305115

Best val_accuracy So Far: 0.9133333563804626
Total elapsed time: 00h 05m 26s
INFO:tensorflow:Oracle triggered exit


In [35]:
final_model = tuner_search.get_best_models(num_models=1)[0]

In [37]:
final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1769536   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,798,522
Trainable params: 1,798,522
Non-trainable params: 0
_________________________________________________________________


In [41]:
best_hp = tuner_search.get_best_hyperparameters(num_trials=1)[0]

In [42]:
best_hp.values

{'conv_1_filter': 64,
 'conv_1_kernel': 3,
 'conv_2_filter': 48,
 'conv_2_kernel': 3,
 'dense_1_units': 64,
 'learning_rate': 0.001}